In [1]:
# CS 4440 - Amazon dataset embeddings
# Adapted from https://towardsdatascience.com/building-a-recommendation-system-using-neural-network-embeddings-1ef92e5c80c9

import keras
import pandas as pd
import gzip
import json
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

In [3]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [195]:
meta_df = getDF('meta_Prime_Pantry.json.gz')
rows,columns=meta_df.shape
print('Number of rows: ',rows)
print('Number of columns: ',columns)
meta_df.head()

Number of rows:  10815
Number of columns:  11


,description,title,also_buy,image,brand,details,price,asin,also_view,rank,feature
0,[Sink your sweet tooth into MILK DUDS Candya d...,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",[https://images-na.ssl-images-amazon.com/image...,Milk Duds,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",$5.00,B00005BPJO,NaN,NaN,NaN
1,[Sink your sweet tooth into MILK DUDS Candya d...,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",[https://images-na.ssl-images-amazon.com/image...,Milk Duds,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",$5.00,B00005BPJO,NaN,NaN,NaN
2,[A perfect Lentil soup starts with Goya Lentil...,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",[https://images-na.ssl-images-amazon.com/image...,Goya,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,B0000DIF38,"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...",NaN,NaN
3,[Saran Premium Wrap is an extra tough yet easy...,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",[https://images-na.ssl-images-amazon.com/image...,Saran,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,B0000DIWNI,"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...",NaN,NaN
4,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",NaN,[https://images-na.ssl-images-amazon.com/image...,Saran,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,B0000DIWNZ,[B0014CZ0TE],NaN,NaN


In [196]:
#Number of unique user id  in the data
print('Number of unique product in Raw data = ', meta_df['asin'].nunique())

Number of unique product in Raw data =  10814


In [197]:
df = meta_df[['asin', 'also_buy', 'also_view', 'title']]

In [198]:
df.head()

,asin,also_buy,also_view,title
0,B00005BPJO,"[B019KE37WO, B007NQSWEU]",NaN,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C..."
1,B00005BPJO,"[B019KE37WO, B007NQSWEU]",NaN,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C..."
2,B0000DIF38,"[B003SI144W, B000VDRKEK]","[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...","Goya Dry Lentils, 16 oz"
3,B0000DIWNI,"[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...","[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...","Saran Premium Plastic Wrap, 100 Sq Ft"
4,B0000DIWNZ,NaN,[B0014CZ0TE],"Saran Cling Plus Plastic Wrap, 200 Sq Ft"


In [199]:
# Remove asin if also_buy and also_view are both NaN
df = df.dropna(subset=['also_buy', 'also_view'], how='all')

#Remove duplicates asin
df.drop_duplicates(subset ="asin", keep = 'first', inplace = True) 

In [200]:
df.shape

(7784, 4)

In [201]:
df.loc[df['also_buy'].isnull(),['also_buy']] = df.loc[df['also_buy'].isnull(),'also_buy'].apply(lambda x: [])
df.loc[df['also_view'].isnull(),['also_view']] = df.loc[df['also_view'].isnull(),'also_view'].apply(lambda x: [])
df['buy_and_view'] = df['also_buy'] + df['also_view']

In [202]:
df.shape
df.head()

,asin,also_buy,also_view,title,buy_and_view
0,B00005BPJO,"[B019KE37WO, B007NQSWEU]",[],"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]"
2,B0000DIF38,"[B003SI144W, B000VDRKEK]","[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...","Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK, B074MFVZG7, B079PTH69..."
3,B0000DIWNI,"[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...","[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...","Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8..."
4,B0000DIWNZ,[],[B0014CZ0TE],"Saran Cling Plus Plastic Wrap, 200 Sq Ft",[B0014CZ0TE]
5,B0000GH6UG,[B008MHKWLK],[],"Ibarra Mexican Chocolate, 19 oz",[B008MHKWLK]


In [203]:
# Mapping of asin to index and index to asin
asin_index = {}
index_asin = {}

for i, a in enumerate(df['asin']):
    asin_index[a] = i
    index_asin[i] = a

link_index = {}
index_link = {}
index = 0 
# add the asin for buy_and_view
for link_arr in df['buy_and_view']:
    for link in link_arr:
        if link not in link_index:
            link_index[link] = index
            index_link[index] = link
            index += 1

In [204]:
pairs = []
asin_name = {}
for index, row in df.iterrows():
    asinp = row['asin']
    namep = row['title']
    asin_name[asinp] = namep
    for link in row['buy_and_view']:
        pairs.append((asin_index[row['asin']], link_index[link]))

In [161]:
import numpy as np
import random
random.seed(100)

def generate_batch(pairs, n_positive = 50, negative_ratio = 1.0, classification = False):
    """Generate batches of samples for training"""
    batch_size = n_positive * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    pairs_set = set(pairs)
    
    # Adjust label based on task
    if classification:
        neg_label = 0
    else:
        neg_label = -1
    
    # This creates a generator
    while True:
        # randomly choose positive examples
        for idx, (asin_id, link_id) in enumerate(random.sample(pairs, n_positive)):
            batch[idx, :] = (asin_id, link_id, 1)

        # Increment idx by 1
        idx += 1
        
        # Add negative examples until reach batch size
        while idx < batch_size:
            
            # random selection
            random_asin = random.randrange(len(asin_index))
            random_link = random.randrange(len(link_index))
            
            # Check to make sure this is not a positive example
            if (random_asin, random_link) not in pairs_set:
                
                # Add to batch and increment index
                batch[idx, :] = (random_asin, random_link, neg_label)
                idx += 1
                
        # Make sure to shuffle order
        np.random.shuffle(batch)
        yield {'asin': batch[:, 0], 'link': batch[:, 1]}, batch[:, 2]

In [162]:
next(generate_batch(pairs, n_positive = 2, negative_ratio = 2))

({'asin': array([2224., 3219., 2865., 1431., 3726., 6274.]),
  'link': array([ 3671., 11992.,  7102., 11559., 12628.,  8076.])},
 array([ 1., -1., -1., -1., -1.,  1.]))

In [166]:
from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model
def embedding_model(embedding_size = 50, classification = False):
    """Model to embed item and link using the Keras functional API.
       Trained to discern if another product is in the corresponding list"""
    
    # Both inputs are 1-dimensional
    asin = Input(name = 'asin', shape = [1])
    link = Input(name = 'link', shape = [1])
    
    # Embedding the asin (shape will be (None, 1, 50))
    asin_embedding = Embedding(name = 'asin_embedding',
                               input_dim = len(asin_index),
                               output_dim = embedding_size)(asin)
    
    # Embedding the link (shape will be (None, 1, 50))
    link_embedding = Embedding(name = 'link_embedding',
                               input_dim = len(link_index),
                               output_dim = embedding_size)(link)
    
    # Merge the layers with a dot product along the second axis 
    # (shape will be (None, 1, 1))
    merged = Dot(name = 'dot_product', normalize = True, 
                 axes = 2)([asin_embedding, link_embedding])
    
    # Reshape to be a single number (shape will be (None, 1))
    merged = Reshape(target_shape = [1])(merged)
    
    # Squash outputs for classification
    out = Dense(1, activation = 'sigmoid')(merged)
    model = Model(inputs = [asin, link], outputs = out)
    
    # Compile using specified optimizer and loss 
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', 
                  metrics = ['accuracy'])
    
    return model

In [167]:
model = embedding_model()
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
asin (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
link (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
asin_embedding (Embedding)      (None, 1, 50)        389200      asin[0][0]                       
__________________________________________________________________________________________________
link_embedding (Embedding)      (None, 1, 50)        760600      link[0][0]                       
____________________________________________________________________________________________

In [242]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 15

def find_similar(name, weights, index_name = 'asin', n = 10, least = False, return_dist = False, plot = False):
    """Find n most similar items (or least) to name based on embeddings. Option to also plot the results"""
    
    # Select index and reverse index
    if index_name == 'asin':
        index = asin_index
        rindex = index_asin
    elif index_name == 'link':
        index = link_index
        rindex = index_link
    
    # Check to make sure `name` is in index
    try:
        # Calculate dot product between book and all others
        dists = np.dot(weights, weights[index[name]])
    except KeyError:
        print(f'{name} Not Found.')
        return
    
    # Sort distance indexes from smallest to largest
    sorted_dists = np.argsort(dists)
    
    # Plot results if specified
    if plot:
        
        # Find furthest and closest items
        furthest = sorted_dists[:(n // 2)]
        closest = sorted_dists[-n-1: len(dists) - 1]
        items = [rindex[c] for c in furthest]
        items.extend(rindex[c] for c in closest)
        
        # Find furthest and closets distances
        distances = [dists[c] for c in furthest]
        distances.extend(dists[c] for c in closest)
        
        colors = ['r' for _ in range(n //2)]
        colors.extend('g' for _ in range(n))
        
        data = pd.DataFrame({'distance': distances}, index = items)
        
        # Horizontal bar chart
        data['distance'].plot.barh(color = colors, figsize = (10, 8),
                                   edgecolor = 'k', linewidth = 2)
        plt.xlabel('Cosine Similarity');
        plt.axvline(x = 0, color = 'k');
        
        # Formatting for italicized title
        name_str = f'{index_name.capitalize()}s Most and Least Similar to'
        for word in name.split():
            # Title uses latex for italize
            name_str += ' $\it{' + word + '}$'
        plt.title(name_str, x = 0.2, size = 28, y = 1.05)
        
        return None
    
    # If specified, find the least similar
    if least:
        # Take the first n from sorted distances
        closest = sorted_dists[:n]
         
        print(f'{index_name}s furthest from {asin_name[name]} ({name}).\n')
        
    # Otherwise find the most similar
    else:
        # Take the last n sorted distances
        closest = sorted_dists[-n:]
        
        # Need distances later on
        if return_dist:
            return dists, closest
        
        
        print(f'{index_name.capitalize()}s closest to {asin_name[name]} ({name}).\n')
        
    # Need distances later on
    if return_dist:
        return dists, closest
    
    
    # Print formatting
    max_width = max([len(rindex[c]) for c in closest])
    
    # Print the most similar and distances
    for c in reversed(closest):
        print(f'''{asin_name[rindex[c]]:{max_width + 2}} 
        {index_name.capitalize()}: {rindex[c]:{max_width + 2}} Similarity: {dists[c]:.{2}}''')

In [287]:
n_positive = 2**12
epochs = 30
negative_ratio = 1
validation_split = 0.2

gen = generate_batch(pairs, n_positive, negative_ratio = negative_ratio)

steps_per_epoch = len(pairs) // n_positive


# Train
h = model.fit_generator(gen, epochs = epochs, 
                        steps_per_epoch = steps_per_epoch)

Epoch 1/30
73/73 [==============================] - 4s 51ms/step - loss: -6.0654 - accuracy: 0.3592
Epoch 2/30
46/73 [=================>............] - ETA: 1s - loss: -5.8920 - accuracy: 0.3754

KeyboardInterrupt: 

In [280]:
model.save(f'models/model-n_positive-{n_positive}-epochs-{epochs}-negative_ratio-{negative_ratio}.h5')

In [281]:
asin_layer = model.get_layer('asin_embedding')
asin_weights = asin_layer.get_weights()[0]
asin_weights.shape

(7784, 50)

In [282]:
asin_weights = asin_weights / np.linalg.norm(asin_weights, axis = 1).reshape((-1, 1))
asin_weights[0][:10]
np.sum(np.square(asin_weights[0]))

1.0000001

In [285]:
prod_lst = ['B0000DIWNZ, B0000GH6UG', 'B0000DIWNI', 'B00005BPJO', 'B01GCO3KDQ', 'B00UOLAPB4']
for prod in prod_lst:
    find_similar(prod, asin_weights)
    print("\n\n")

B0000DIWNZ, B0000GH6UG Not Found.



Asins closest to Saran Premium Plastic Wrap, 100 Sq Ft (B0000DIWNI).

Saran Premium Plastic Wrap, 100 Sq Ft 
        Asin: B0000DIWNI   Similarity: 1.0
CeraVe Hydrating Body Wash 10 oz 
        Asin: B01CTRKAQU   Similarity: 1.0
Poise® Incontinence Pads, Maximum Absorbency, Long, 39 Count 
        Asin: B0166NBALC   Similarity: 1.0
Neutrogena Ultra Gentle Daily Facial Cleanser for Sensitive Skin, Oil-Free, Soap-Free, Hypoallergenic & Non-Comedogenic Foaming Face Wash, 12 fl. oz 
        Asin: B00UOLD4DK   Similarity: 0.99
Ntg Makeup Rmvr Clnsng Twlt 25s Rfl Twpk 
        Asin: B0182G7AQ0   Similarity: 0.99
Cheez-It Baked Snack Cheese Crackers, White Cheddar, 7 oz Box 
        Asin: B013JMERZ4   Similarity: 0.99
CeraVe Eczema Soothing Body Wash 10 oz with Omega Oils and Ceramides for Cleansing and Calming Dry, Itchy, Eczema Prone Skin 
        Asin: B01BONXOWW   Similarity: 0.98
Health Warrior Chia Bars Variety Pack, Coconut/Acai Berry/Mango, 15 Coun